# Asset Quality Matrix (AQM)

In [91]:
import numpy as np
import pandas as pd
import yfinance as yf

Drawing Inspiration from the Beta Ratio and how the FDIC measures Asset quality, I believe that the most suitable way to measure asset quality is to evaluate the deviations of returns of a particular asset against a market index. In this notebook, I use the Treasury Inflation Protected Securities (TIPS) as the market index which is essentially treasury bonds adjusted for inflation.

We measure the asset quality of the asset with the following equation:


#### $\text{Asset Quality =} \frac{\text{Covariance(Returns(Asset), Returns(Market))}}{\text{Variance(Returns(Market))}}$

In [92]:
tip = yf.Ticker("TIP")
tip_data = tip.history(period="5y")  
tip_data = tip_data[['Close']]
tip_data.index = tip_data.index.tz_localize(None)
tip_data['returns'] = tip_data['Close'].pct_change().dropna()
tip_data = tip_data.iloc[1:] 

market_var = np.var(tip_data['returns'])

tip_data

,Close,returns
Date,,
2020-05-27,101.667236,0.000990
2020-05-28,101.985550,0.003131
2020-05-29,102.027458,0.000411
2020-06-01,102.153107,0.001232
2020-06-02,101.926903,-0.002214
...,...,...
2025-05-16,108.879997,0.001748
2025-05-19,108.889999,0.000092
2025-05-20,108.730003,-0.001469


### Below is the calculation for blue-chip equities listed on the S&P 500

In [93]:
sp500 = yf.Ticker("^GSPC")
sp500_data = sp500.history(period="5y")
sp500_data = sp500_data[['Close']]
sp500_data.index = sp500_data.index.tz_localize(None)
sp500_data['returns'] = sp500_data['Close'].pct_change().dropna()
sp500_data = sp500_data.iloc[1:]
sp500_covar = np.cov(sp500_data['returns'], tip_data['returns'])[0][1]
sp500_aqm = sp500_covar / market_var

sp500_data

,Close,returns
Date,,
2020-05-27,3036.129883,0.014827
2020-05-28,3029.729980,-0.002108
2020-05-29,3044.310059,0.004812
2020-06-01,3055.729980,0.003751
2020-06-02,3080.820068,0.008211
...,...,...
2025-05-16,5958.379883,0.007005
2025-05-19,5963.600098,0.000876
2025-05-20,5940.459961,-0.003880


In [94]:
print(f'The aqm of the sp500 is {sp500_aqm:.2f}')

The aqm of the sp500 is 0.52


### Below is the calculation for an example of a mid-cap equity defined by Investopedia as an equity with a market cap between $2 - $10 Billion

Mid-Cap equity: Avis Budget Group Inc
Ticker: CAR

In [95]:
car = yf.Ticker("CAR")
car_data = car.history(period="5y")
car_data = car_data[['Close']]
car_data.index = car_data.index.tz_localize(None)
car_data['returns'] = car_data['Close'].pct_change().dropna()
car_data = car_data.iloc[1:]
car_covar = np.cov(car_data['returns'], tip_data['returns'])[0][1]
car_aqm = car_covar / market_var

car_data

,Close,returns
Date,,
2020-05-27,20.873856,0.180548
2020-05-28,19.021149,-0.088757
2020-05-29,20.455811,0.075425
2020-06-01,23.258625,0.137018
2020-06-02,22.536545,-0.031046
...,...,...
2025-05-16,108.839996,0.029999
2025-05-19,112.720001,0.035649
2025-05-20,112.000000,-0.006388


In [96]:
print(f'The aqm of Avis Inc is {car_aqm:.2f}')

The aqm of Avis Inc is 0.89


### Below is the calculation for an example of a low-cap equity defined by Investopedia as an equity with a market cap between $250million - $2 Billion

Mid-Cap equity: Defi Development Corp
Ticker: DFDV

In [97]:
DFDV = yf.Ticker("DFDV")
DFDV_data = DFDV.history(period="1y")
DFDV_data = DFDV_data[['Close']]
DFDV_data.index = DFDV_data.index.tz_localize(None)
DFDV_data['returns'] = DFDV_data['Close'].pct_change()
DFDV_data = DFDV_data.dropna()
DFDV_data = DFDV_data.iloc[1:]
tip_data_DFDV = tip_data.loc[DFDV_data.index]
market_var_1 = np.var(tip_data_DFDV['returns'])
DFDV_covar = np.cov(DFDV_data['returns'], tip_data_DFDV['returns'])[0][1]
DFDV_aqm = DFDV_covar / market_var_1

DFDV_data

,Close,returns
Date,,
2024-05-28,1.200000,-0.036697
2024-05-29,1.165714,-0.028572
2024-05-30,1.131429,-0.029411
2024-05-31,1.120000,-0.010101
2024-06-03,1.131429,0.010204
...,...,...
2025-05-16,22.427143,0.744527
2025-05-19,18.114286,-0.192305
2025-05-20,25.485714,0.406940


In [98]:
print(f'The aqm of Defi Development Corp is {DFDV_aqm:.2f}')

The aqm of Defi Development Corp is -48.81


### Below is the calculation for an example of a blue-chip cryptocurrency: Ethereum

In [106]:
eth = yf.Ticker("ETH-USD")
eth_data = eth.history(period="1y")[['Close']]
eth_data.index = eth_data.index.tz_localize(None)
eth_data['returns'] = eth_data['Close'].pct_change()
eth_data = eth_data.dropna()

common_dates = eth_data.index.intersection(tip_data.index)
eth_data_aligned = eth_data.loc[common_dates]
tip_data_aligned = tip_data.loc[common_dates]

tip_var = np.var(tip_data_aligned['returns'])
eth_tip_covar = np.cov(eth_data_aligned['returns'], tip_data_aligned['returns'])[0][1]

eth_aqm = eth_tip_covar / tip_var



In [107]:
print(f'The aqm of ETH is {eth_aqm:.2f}')

The aqm of ETH is 0.85


### Below is the calculation for an example of a mid-cap cryptocurrency: Avalanche

In [108]:
AVAX = yf.Ticker("AVAX-USD")
AVAX_data = AVAX.history(period="1y")[['Close']]
AVAX_data.index = AVAX_data.index.tz_localize(None)
AVAX_data['returns'] = AVAX_data['Close'].pct_change()
AVAX_data = AVAX_data.dropna()

common_dates = AVAX_data.index.intersection(tip_data.index)
AVAX_data_aligned = AVAX_data.loc[common_dates]
tip_data_aligned = tip_data.loc[common_dates]

tip_var = np.var(tip_data_aligned['returns'])
AVAX_tip_covar = np.cov(AVAX_data_aligned['returns'], tip_data_aligned['returns'])[0][1]

AVAX_aqm = AVAX_tip_covar / tip_var


In [109]:
print(f'The aqm of AVAX is {AVAX_aqm:.2f}')

The aqm of AVAX is -0.13


### Below is the calculation for an example of a blue-chip memecoin: Dogecoin

In [110]:
doge = yf.Ticker("DOGE-USD")
doge_data = doge.history(period="1y")[['Close']]
doge_data.index = doge_data.index.tz_localize(None)
doge_data['returns'] = doge_data['Close'].pct_change()
doge_data = doge_data.dropna()

common_dates = doge_data.index.intersection(tip_data.index)
doge_data_aligned = doge_data.loc[common_dates]
tip_data_aligned = tip_data.loc[common_dates]

tip_var = np.var(tip_data_aligned['returns'])
doge_tip_covar = np.cov(doge_data_aligned['returns'], tip_data_aligned['returns'])[0][1]

doge_aqm = doge_tip_covar / tip_var

In [111]:
print(f'The aqm of doge is {doge_aqm:.2f}')

The aqm of doge is -0.77
